In [1]:
# 1) Imports e configuração básica
import numpy as np
from pathlib import Path
import pandas as pd

In [2]:
# 1) Carregar dataset de gols (goal scorers) e preparar colunas-base
PROJECT_ROOT = Path.cwd().parent
DATA_RAW = PROJECT_ROOT / "data"

# Ajuste o caminho conforme onde o arquivo está no seu projeto
goals = pd.read_csv(DATA_RAW / "goalscorers.csv", parse_dates=["date"])

# Coluna "copa" (ano do jogo)
goals["copa"] = goals["date"].dt.year

# Padronização mínima (evita problemas de espaços)
goals["scorer"] = goals["scorer"].astype(str).str.strip()
goals["team"] = goals["team"].astype(str).str.strip()

# 3) Remover gols contra (não representam "decisão" de um jogador)
if "own_goal" in goals.columns:
    goals = goals[goals["own_goal"].fillna(False).eq(False)].copy()

# # 4) (Opcional) manter duas visões: com e sem pênalti (útil pra robustez)
# if "penalty" in goals.columns:
#     goals_no_pen = goals[goals["penalty"].fillna(False).eq(False)].copy()
# else:
#     goals_no_pen = goals.copy()

# print("Linhas (com pênalti):", len(goals))
# print("Linhas (sem pênalti):", len(goals_no_pen))
display(goals.head())
goals.shape


,date,home_team,away_team,team,scorer,minute,own_goal,penalty,copa
0,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,44.0,False,False,1916
1,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,55.0,False,False,1916
2,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,70.0,False,False,1916
3,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,75.0,False,False,1916
4,1916-07-06,Argentina,Chile,Argentina,Alberto Ohaco,2.0,False,False,1916


(40389, 9)

In [4]:
# 2) Filtrar gols apenas para partidas de Copa do Mundo usando o results.csv como fonte da verdade
PROJECT_ROOT = Path.cwd().parent
DATA_RAW = PROJECT_ROOT / "data"
results = pd.read_csv(DATA_RAW / "results.csv", parse_dates=["date"])

# só jogos de Copa do Mundo
# (ajuste o texto se no seu dataset estiver diferente, ex: "FIFA World Cup")
wc_results = results[results["tournament"].eq("FIFA World Cup")].copy()

# Criar uma "chave" do jogo para cruzar com goal_scorers
# (date + home_team + away_team)
wc_results["match_key"] = (
    wc_results["date"].dt.strftime("%Y-%m-%d") + " | " +
    wc_results["home_team"].astype(str).str.strip() + " | " +
    wc_results["away_team"].astype(str).str.strip()
)

goals["match_key"] = (
    goals["date"].dt.strftime("%Y-%m-%d") + " | " +
    goals["home_team"].astype(str).str.strip() + " | " +
    goals["away_team"].astype(str).str.strip()
)

# Filtrar: manter só gols cujas partidas existem em wc_results
wc_goals = goals[goals["match_key"].isin(set(wc_results["match_key"]))].copy()

# Criar coluna "copa" agora no dataset já filtrado
wc_goals["copa"] = wc_goals["date"].dt.year

print("Jogos de Copa (results):", len(wc_results))
print("Gols em jogos de Copa (goal_scorers filtrado):", len(wc_goals))
display(wc_goals)


Jogos de Copa (results): 964
Gols em jogos de Copa (goal_scorers filtrado): 2666


,date,home_team,away_team,team,scorer,minute,own_goal,penalty,copa,match_key
292,1930-07-13,Belgium,United States,United States,Bart McGhee,23.0,False,False,1930,1930-07-13 | Belgium | United States
293,1930-07-13,Belgium,United States,United States,Tom Florie,45.0,False,False,1930,1930-07-13 | Belgium | United States
294,1930-07-13,Belgium,United States,United States,Bert Patenaude,69.0,False,False,1930,1930-07-13 | Belgium | United States
295,1930-07-13,France,Mexico,France,Lucien Laurent,19.0,False,False,1930,1930-07-13 | France | Mexico
296,1930-07-13,France,Mexico,France,Marcel Langiller,40.0,False,False,1930,1930-07-13 | France | Mexico
...,...,...,...,...,...,...,...,...,...,...
41003,2022-12-18,Argentina,France,Argentina,Ángel Di María,36.0,False,False,2022,2022-12-18 | Argentina | France
41004,2022-12-18,Argentina,France,France,Kylian Mbappé,80.0,False,True,2022,2022-12-18 | Argentina | France
41005,2022-12-18,Argentina,France,France,Kylian Mbappé,81.0,False,False,2022,2022-12-18 | Argentina | France
41006,2022-12-18,Argentina,France,Argentina,Lionel Messi,109.0,False,False,2022,2022-12-18 | Argentina | France


In [ ]:
gols_por_jogador = (
    wc_goals
        .groupby(["copa", "scorer", "team"])
        .size()
        .reset_index(name="gols")
        .sort_values(["copa", "gols"], ascending=[True, False])
)


,Copa,Artilheiro,Seleção,Gols
12,1930,Guillermo Stábile,Argentina,8
29,1930,Pedro Cea,Uruguay,5
6,1930,Bert Patenaude,United States,4
8,1930,Carlos Peucelle,Argentina,3
16,1930,Ivan Bek,Yugoslavia,3
...,...,...,...,...
1636,2022,Xherdan Shaqiri,Switzerland,1
1637,2022,Yeltsin Tejeda,Costa Rica,1
1639,2022,Zakaria Aboukhlal,Morocco,1
1641,2022,Ángel Di María,Argentina,1


In [14]:
# Manter apenas o artilheiro (top 1) de cada Copa

artilheiro_por_copa = (
    gols_por_jogador
        .groupby("copa")
        .head(1)
        .reset_index(drop=True)
)

display(artilheiro_por_copa.rename(columns={
        "copa": "Copa",
        "scorer": "Artilheiro",
        "team": "Seleção",
        "gols": "Gols"
    })
)


,Copa,Artilheiro,Seleção,Gols
0,1930,Guillermo Stábile,Argentina,8
1,1934,Oldřich Nejedlý,Czechoslovakia,5
2,1938,Leônidas da Silva,Brazil,7
3,1950,Ademir de Menezes,Brazil,9
4,1954,Sándor Kocsis,Hungary,11
5,1958,Just Fontaine,France,13
6,1962,Dražan Jerković,Yugoslavia,4
7,1966,Eusébio,Portugal,9
8,1970,Gerd Müller,Germany,10
9,1974,Grzegorz Lato,Poland,7


In [15]:
gols_por_jogador["rank"] = (
    gols_por_jogador
        .groupby("copa")["gols"]
        .rank(method="first", ascending=False)
)

top5_por_copa = gols_por_jogador[gols_por_jogador["rank"] <= 5].copy()

In [16]:
# Total de gols por Copa
total_gols_por_copa = (
    wc_goals
        .groupby("copa")
        .size()
        .reset_index(name="total_gols")
)

# Gols somados do Top 5 por Copa
gols_top5_por_copa = (
    top5_por_copa
        .groupby("copa")["gols"]
        .sum()
        .reset_index(name="gols_top5")
)

# Juntar tudo e calcular o percentual
share_top5 = (
    total_gols_por_copa
        .merge(gols_top5_por_copa, on="copa")
)

share_top5["share_top5_%"] = (
    share_top5["gols_top5"] / share_top5["total_gols"] * 100
)

display(share_top5.round(2))


,copa,total_gols,gols_top5,share_top5_%
0,1930,69,23,33.33
1,1934,70,19,27.14
2,1938,82,26,31.71
3,1950,88,26,29.55
4,1954,136,33,24.26
5,1958,126,35,27.78
6,1962,89,20,22.47
7,1966,87,27,31.03
8,1970,94,30,31.91
9,1974,94,25,26.60


In [20]:
share_print = share_top5.rename(columns={
    "copa": "Copa",
    "total_gols": "Total de gols",
    "gols_top5": "Gols do Top 5",
    "share_top5_%": "% de gols concentrados no Top 5"
})

(
    share_print
        .style
        .format({"% de gols concentrados no Top 5": "{:.2f}%"})
        .background_gradient(subset=["% de gols concentrados no Top 5"], cmap="Reds")
        .set_caption("Proporção de gols concentrada nos cinco principais artilheiros de cada Copa do Mundo")
)


,Copa,Total de gols,Gols do Top 5,% de gols concentrados no Top 5
0,1930,69,23,33.33%
1,1934,70,19,27.14%
2,1938,82,26,31.71%
3,1950,88,26,29.55%
4,1954,136,33,24.26%
5,1958,126,35,27.78%
6,1962,89,20,22.47%
7,1966,87,27,31.03%
8,1970,94,30,31.91%
9,1974,94,25,26.60%
